Importing Packages And Basic Set up

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from plotnine import *

import numpy as np
import pandas as pd

np.random.seed(23)

Importing Datasets

In [ ]:
# Importing maindata

file_path = "C://Users/miste/Documents/Causal_ML/"

x = pd.read_stata(file_path + "maindata.dta", convert_categoricals=False)

# Importing laws_csv, cleaning it
laws_csv = pd.read_csv("When_Were_Laws.csv")
laws_csv = laws_csv[np.logical_not(np.isnan(laws_csv["FIPS"]))]  # FIPS codes identify states
laws_csv = laws_csv.drop("State_Name", axis=1)  # Dropping as useless
laws_csv = laws_csv.rename({'FIPS': 'stfips'}, axis=1) 

# Merging
merged = pd.merge(laws_csv, x, on='stfips', how='outer')

Cleaning Datasets, only interested in the 1997 states. 

In [ ]:
basic_merged = merged.copy()  # To allow for re-running 

basic_merged = basic_merged[basic_merged["a_age"] <= 25]  # Can be changed later, but for now useful I think
#age_subset = np.logical_and(np.greater_equal(basic_merged["a_age"],18), np.greater_equal(19,basic_merged["a_age"]))
# 17 <= age <= 21 (maybe should be like 22)
#basic_merged = basic_merged[age_subset]
#print(basic_merged.shape)

# Dropping states who were treated < 97 (i.e. they always had programs)
# This is following Callaway + Sant'anna, as we cannot meaningfully 
# do any inference using those states. Although we can compare them later as a 
# robustness check, which may be interesting
basic_merged = basic_merged[basic_merged["Year_Implemented"].str.contains("always")==False]  

# Setting never implemented states to implementing at t=inf
basic_merged = basic_merged.replace("never", "1000000") 
basic_merged["Year_Implemented"] = basic_merged["Year_Implemented"].astype(int)  # converting to intbasic_merged = basic_merged[basic_merged["Year_Implemented"].str.contains("never")==False]  # Only want to look at one for now. 

# Drop Arizona since they implemented late and later repealed policy
basic_merged = basic_merged[basic_merged["stfips"] != 5]

# As we are treating >19 as the never-treated group, we set their year implemented as 1000000 >> 1999
year_implemented_vector = basic_merged["Year_Implemented"].copy()
year_implemented_vector[basic_merged["under19"] == 0] = 1000000
basic_merged["group"] = year_implemented_vector  # Equals the year you were first treated. If >=19 then treated at t = infty

# Generating list of confounders of interest, these are not necessarily optimal. 
list_of_confounders = ["fownu18", "a_maritl", "female" , "povll"]
list_of_confounders += ["anykids", "disability", "collgrad", "hsgrad"]

$\underline{\bold{\text{Estimating causal effect}}}$

Fitting Model

In [ ]:
def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, year_of_interest:int):
    '''
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each set of covariates in fold j
    3b. Each set of covariates is fitted for each combination of year x treatment
    4. We return a matrix which contains those predictions as columns (each column is one combinatino of year x treatment)
    5. We also return vectors which indicate the true year/treatment status for that particular set of covariates
    

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    year_of_interest: year that treatment occured
    '''
    # Setting up prediction vectors
    # the 2*i'th column is year `i` with treatment = 0
    # the (2*i)+1'th column is year `i` with treatment = 1
    # As we predict for 18 years, there are 36 columns. 

    predictions_matrix = np.empty(shape=(y.shape[0], 36))

    # k-folding
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # Generating vectors which will contain the true treatment/year combination for each covariate. 
    year_vec = np.full_like(X_w_treatment["year"], np.nan, dtype=float)
    treat_vec = np.full_like(X_w_treatment["year"], np.nan, dtype=float)

    for train_index, test_index in kf.split(X_w_treatment, y):
        X_train = X_w_treatment.loc[train_index]
        y_train = y.loc[train_index]
        q = make_model()
        q.fit(X_train, y_train)

        # This saves the vector of years so we only get the att for the 1997 group
        year_vec[test_index] =  X_w_treatment["year"].loc[test_index]

        # This saves the vector of treated statuses, as we want to condition on A = 1 (we want ATT, not ATE)
        treat_vec[test_index] = A.loc[test_index]

        for j in range(1991, 2009):
            X0 = X_w_treatment.copy()
            X0["A"] = 1000000  # Never treated units. 
            X0["year"] = j # Changing the year to the year of interest
            col_num = 2*(j - 1991)  # Which column the data should go in
            predictions_matrix[test_index,col_num] = q.predict_proba(X0.loc[test_index])[:, 1]
        for j in range(1991,2009):
            X1 = X_w_treatment.copy() 
            # We do not adjust the year here, as we are only interested in their actual treatment status
            # That means we don't need to change it, as it is already how we want it. 
            X1["year"] = j # Changing the year to the year of interest
            col_num = 2*(j - 1991) + 1  # Which column the data should go in
            predictions_matrix[test_index,col_num] = q.predict_proba(X1.loc[test_index])[:, 1]

    assert np.isnan(predictions_matrix).sum() == 0  # Sanity check that we have no missings
    return predictions_matrix, year_vec, treat_vec

def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int, year_of_interest:int):
    """
    k-fold predicting A given X

    Inputs: 
    make_model: G_model of interest
    X: Dataframe (always basic_merged)
    A: vector of which group each person belongs to
    n_splits: how much we split for the k-folding
    year_of_interest: what group we are estimating at this point. 
    """

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # group of people 
    G_g_and_C = np.logical_or(np.equal(A,year_of_interest), np.equal(A,1000000))

    # subset for people in G_g or C, then reset index to allow for kfolding
    covariates = X.copy()
    covariates = covariates.loc[G_g_and_C]
    covariates = covariates.reset_index(drop=True)

    treatment_status = (A == year_of_interest)
    treatment_status = treatment_status.loc[G_g_and_C]
    treatment_status = treatment_status.reset_index(drop=True)

    g_matrix = np.full_like(treatment_status, np.nan, dtype=float)

    # Generating vectors which will contain the true treatment/year combination for each covariate. 
    # Normalizing years so that year =1 when treatment first occurs. 
    #covariates.drop("year", axis=1)  # We are interested in the covariates, not the year they were found in. 

    for train_index, test_index in kf.split(covariates, treatment_status):
        X_train = covariates.loc[train_index]
        y_train = treatment_status.loc[train_index]
        g = make_model()
        g.fit(X_train, y_train)
        g_matrix[test_index] = g.predict_proba(covariates.loc[test_index])[:, 1]
    assert np.isnan(g_matrix).sum() == 0  # Sanity check that we have no missings
    print("Max p:" + str(np.max(g_matrix))) # Checking for overlap
    return g_matrix

Estimating the ATT's


In [ ]:
def ATT_event_study(year_of_interest:int, response:str, confounder_list:list, dataframe:pd.DataFrame, G_model, Q_model):
    '''
    Estimates ATT(g,t) for the combinations of g,t which we are interested in. 

    Based on Callaway & Sant'anna, we compute ATT(g,t) over time

    Inputs: 
    year_of_interest: the year at which treatment first occurs
    response: which outcome we are interested in, out of public/private/all insurance. 
    confounder_list: list of columns of dataframe to use as confounders
    dataframe: dataframe to get data from
    g_model: machine learning ensemble model to use as the g model
    q_model : machine learning ensemble model to use as the q model
    '''
    # We don't subset here, as we want to fit on all data. 
    #state_level = dataframe.loc[dataframe['stfips'].isin(stfips),:] # Subset for states of interest
    results_df = {}
    state_level = dataframe.copy()
    state_level = state_level[confounder_list+ ["group","year", response]]  # Subsetting for features of interest

    # Drop missings, only needed for some models, 
    state_level = state_level.dropna(axis = 0)

    # Setting up 
    state_level = state_level.reset_index(drop=True)
    confounders = state_level[list_of_confounders + ["year"]]
    treatment = state_level["group"]
    outcome = state_level[response]

    predictions_matrix, year_vec, treat_vec = outcome_k_fold_fit_and_predict(Q_model, \
                                       X=confounders, y=outcome, A=treatment, n_splits=5, \
                                        year_of_interest=year_of_interest)
                                        
    G_g_and_C = np.logical_or(np.equal(treatment,year_of_interest), np.equal(treatment,1000000))

    predictions_matrix = predictions_matrix[G_g_and_C,:]
    year_vec = year_vec[G_g_and_C]
    treat_vec = treat_vec[G_g_and_C]
    outcome = outcome[G_g_and_C]

    # the 2*i'th column is always the untreated vec, and the (2*i)+1 is treated. 
    g_matrix = treatment_k_fold_fit_and_predict(G_model, X=confounders, A=treatment, n_splits=5, \
                                                year_of_interest=year_of_interest)
                                                
    # Weight on the first term. 
    # This depends only on g so we can pull it out here
    w_g = (treat_vec == year_of_interest) #* (year_vec == year) # We only want the ATT, so we weight appropriately
                                                # to only look at the untils treated at t=1
    w_g = w_g / np.mean(w_g) # To ensure mean of the weights is 1, weights are >= 0 by construction
    
    for i in range(1,18): # we look at the ATT for 1992-2008 (cannot look at 1991 as we have no 1990 data)
        year = i + 1991 # What actual year we are looking at in the for loop
        # Generating term 1:
        m_treat_1 = predictions_matrix[:,(2*i)+1]  # m_treat_1 
        m_nev_1 = predictions_matrix[:,(2*i)]  # m_nev_1
        if year >= year_of_interest: # If we've gotten to the treatment year 
            m_treat_0 = predictions_matrix[:,(2*(year_of_interest-1991))-1]  # m_treat_0
            m_nev_0 = predictions_matrix[:,(2*(year_of_interest-1991))-2]  # m_nev_0
        else:  # otherwise we do a short difference. 
            m_treat_0 = predictions_matrix[:,(2*i)-1]  # m_treat_0
            m_nev_0 = predictions_matrix[:,(2*i)-2]  # m_nev_0
        term_1 = w_g * (m_treat_1 - m_treat_0 - (m_nev_1 - m_nev_0))  

        # Generating term 2:
        # This weight equals 1 if the unit appears at time t and is in group g
        w_treat_1 = (year_vec == year) * (treat_vec == year_of_interest)  # Equals 1 if in year, and treated at year_of_interest

        w_treat_1 = w_treat_1 / np.mean(w_treat_1)

        # This weight equals 1 if the unit appears at time g_d_1 and is in group g
        if year >= year_of_interest: # If we've gotten to the treatment year
            w_treat_0 = (year_vec == year_of_interest - 1) * (treat_vec == year_of_interest)  # Equals 1 if in year, and treated at year_of_interest
            w_treat_0 = w_treat_0 / np.mean(w_treat_0)
        else:
            w_treat_0 = (year_vec == year - 1) * (treat_vec == year_of_interest)  # Equals 1 if in year, and treated at year_of_interest
            w_treat_0 = w_treat_0 / np.mean(w_treat_0)

        term_2 = w_treat_1 * (outcome - m_treat_1) - w_treat_0 * (outcome - m_treat_0)
        
        # Generating term 3:
        w_nev_1 = (year_vec == year) * (treat_vec >= 2002) * g_matrix / (1 - g_matrix)

        w_nev_1 = w_nev_1 / np.mean(w_nev_1)
        
        if year >= year_of_interest:
            w_nev_0 = (year_vec == year_of_interest - 1) * (treat_vec >= 3000) * g_matrix / (1 - g_matrix)
            w_nev_0 = w_nev_0 / np.mean(w_nev_0)
        else:
            w_nev_0 = (year_vec == year - 1) * (treat_vec >= 3000) * g_matrix / (1 - g_matrix)
            w_nev_0 = w_nev_0 / np.mean(w_nev_0)
        
        term_3 = w_nev_1 * (outcome - m_nev_1) - w_nev_0 * (outcome - m_nev_0)

        # Multiplying by 100 to convert to %
        final = term_1 + term_2 - term_3
        mean = np.mean(final) * 100
        phi = final - np.mean(final) * (treat_vec == year_of_interest)
        ste = np.std(phi) * 100 / np.sqrt(final.shape[0])

        results_df.setdefault('year',[]).append(i + 1991)
        results_df.setdefault('att',[]).append(mean)
        results_df.setdefault('ste',[]).append(ste)
        results_df.setdefault('q_model',[]).append(Q_model.__name__ )
        results_df.setdefault('g_model',[]).append(G_model.__name__ )
        results_df.setdefault('response',[]).append(response)
        results_df.setdefault('treat_year',[]).append(year_of_interest)
    
    results_df = pd.DataFrame(results_df)

    return results_df

Define Models to Use

In [ ]:
def large_randf():
    ''' A function that returns a general ML g model for later use in k-folding'''
    return RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)

def xgb():
    ''' A function that returns a general ML q model for later use in k-folding'''
    return XGBClassifier()

In [ ]:
# Getting the ATT's for all years
all_years = pd.DataFrame()

for year in [1997, 1998, 1999]:
    for response in ["pubonly","privonly"]:
        new_model = ATT_event_study(year, response, list_of_confounders, basic_merged, large_randf, xgb)
        all_years = pd.concat([all_years, new_model], ignore_index=True)

all_years['`Implementation Year`'] = all_years["treat_year"].apply(lambda x: str(x))


In [ ]:
all_years.to_csv("./ATT_Results/all_years.csv")
all_years.head()

In [ ]:
# Plotting the pretreatment effects
all_years_from_csv = pd.read_csv("ATT_Results/all_years.csv")
all_years_from_csv['adjusted_year'] = all_years_from_csv["year"] - all_years_from_csv["treat_year"]
all_years_from_csv['`Implementation Year`'] = all_years_from_csv["treat_year"].apply(lambda x: str(x))
all_pre_treat = all_years_from_csv[(all_years_from_csv['adjusted_year'] < 0)].copy()
all_pre_treat['att-error'] = all_pre_treat['att'] - all_pre_treat['ste']
all_pre_treat['att+error'] = all_pre_treat['att'] + all_pre_treat['ste']
all_pre_treat['treat_year'] = all_pre_treat['treat_year'].apply(lambda x: str(x))

p = ggplot(all_pre_treat, aes(x='adjusted_year', y='att', color = '`Implementation Year`', group='`Implementation Year`')) + \
geom_errorbar(aes(ymin = 'att-error', ymax = 'att+error')) + \
geom_point() + \
geom_line() + \
facet_wrap('response') + \
xlab('Years Prior to Treatment') + \
ylab('Average Treatment Effect (percent)') + \
labs(title = "Yearly Average Treatment Effect on the Treated Before Treatment ")

# Save image
p.save(filename = 'pre_treatment_AIPTW_all_cov.png', height=5, width=5, units = 'in', dpi=1000)
p

In [ ]:
# Plotting the post treatment effects
all_years_from_csv = pd.read_csv("ATT_Results/all_years.csv")
all_years_from_csv['adjusted_year'] = all_years_from_csv["year"] - all_years_from_csv["treat_year"]
all_years_from_csv['`Implementation Year`'] = all_years_from_csv["treat_year"].apply(lambda x: str(x))
all_pre_treat = all_years_from_csv[(all_years_from_csv['adjusted_year'] >= 0)].copy()
all_pre_treat['att-error'] = all_pre_treat['att'] - all_pre_treat['ste']
all_pre_treat['att+error'] = all_pre_treat['att'] + all_pre_treat['ste']
all_pre_treat['treat_year'] = all_pre_treat['treat_year'].apply(lambda x: str(x))

p = ggplot(all_pre_treat, aes(x='adjusted_year', y='att', color = '`Implementation Year`', group='`Implementation Year`')) + \
geom_errorbar(aes(ymin = 'att-error', ymax = 'att+error')) + \
geom_point() + \
geom_line() + \
facet_wrap('response') + \
xlab('Years After Treatment') + \
ylab('Average Treatment Effect (percent)') + \
labs(title = "Cumulative Average Treatment Effect on the Treated After Treatment ")

# Save image
p.save(filename = 'post_treatment_AIPTW_all_cov.png', height=5, width=5, units = 'in', dpi=1000)
p